In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
import pandas as pd
import os
from models import columns,vectorize_data
from resultsAnalyse import drawConfusionMatrix
import torch
from sklearn.metrics import f1_score
def filter_inadequada(example):
    return example["INADEQUADA"] == 0

ds = load_dataset("higopires/RePro-categories-multilabel")
print(len(ds["train"]))
#remove INADEQUADA examples
ds = ds.filter(filter_inadequada)
print(len(ds["train"]))

8002
7674


In [2]:


print(ds["train"].features)
#run_blitr(train_df,val_df,test_df)


{'review_text': Value(dtype='string', id=None), 'ENTREGA': Value(dtype='int64', id=None), 'OUTROS': Value(dtype='int64', id=None), 'PRODUTO': Value(dtype='int64', id=None), 'CONDICOESDERECEBIMENTO': Value(dtype='int64', id=None), 'INADEQUADA': Value(dtype='int64', id=None), 'ANUNCIO': Value(dtype='int64', id=None)}


In [1]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to probabilities and then to binary predictions
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    
    return {'f1': float(f1)}

def preprocess_function(sample):
    model_name = "neuralmind/bert-base-portuguese-cased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Tokenize text
    tokenized = tokenizer(sample["review_text"], truncation=True, padding=True, max_length=512, return_tensors="pt")
    
    # Convert binary label columns to a list (e.g., [1, 0, 1, 0, 0, 0])
    labels = []
    for i in range(len(sample["review_text"])):
        label_row = [
            float(sample["ENTREGA"][i]),
            float(sample["OUTROS"][i]),
            float(sample["PRODUTO"][i]),
            float(sample["CONDICOESDERECEBIMENTO"][i]),
            float(sample["ANUNCIO"][i])
        ]
        labels.append(label_row)
    
    tokenized["labels"] = labels
    return tokenized

model_name = "neuralmind/bert-base-portuguese-cased"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5,problem_type="multi_label_classification", ignore_mismatched_sizes=True )



NameError: name 'torch' is not defined

In [ ]:
model.to(device)
tokenized_dataset = ds.map(preprocess_function, batched=True)

In [4]:
from sklearn.metrics import f1_score
small_pred = np.array([[0, 1, 0], [1, 0, 1]])
small_true = np.array([[0, 1, 0], [1, 0, 0]])
print(f1_score(small_true, small_pred, average='samples'))

0.8333333333333333


In [5]:
print(tokenized_dataset["train"]["labels"][2])

[1.0, 0.0, 1.0, 0.0, 0.0]


In [6]:
training_args = TrainingArguments(
    output_dir="./resultsTransformer",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

#trainer.evaluate()

C:\Users\tomas\AppData\Local\Temp\ipykernel_1968\846967199.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.187862,0.902539
2,0.315700,0.132960,0.933729
3,0.157500,0.120323,0.937068
4,0.112800,0.116385,0.939004
5,0.090900,0.116301,0.938117


TrainOutput(global_step=2400, training_loss=0.15434078216552735, metrics={'train_runtime': 2161.4398, 'train_samples_per_second': 17.752, 'train_steps_per_second': 1.11, 'total_flos': 9934552732818492.0, 'train_loss': 0.15434078216552735, 'epoch': 5.0})

In [8]:
y_pred = trainer.predict(tokenized_dataset["test"])
y_pred.metrics

{'test_loss': 0.11935441941022873,
 'test_f1': 0.9335088874259381,
 'test_runtime': 14.293,
 'test_samples_per_second': 67.585,
 'test_steps_per_second': 4.268}

In [ ]:
trainer.save_model("finetuned_portuguese_bert")